In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import lightgbm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
def match_normalize(match):
    match = match.groupby('groupId').mean()
    for head in match.columns.values:
        if head != 'winPlacePerc':
            match[head] /= match[head].max()
    match = match.drop(columns=['maxPlace', 'numGroups'])
    return match
    
    
def rearrange(t_order, p_orders):
    flag = True
    while flag:
        flag = False
        for p_order in p_orders:
            for i in range(len(p_order)-1):
                if t_order.index(p_order[i]) > t_order.index(p_order[i+1]):
                    t_order.remove(p_order[i])
                    t_order.insert(t_order.index(p_order[i+1]), p_order[i])
                    flag = True
    return t_order

In [ ]:
train = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/train_V2.csv').dropna()

In [ ]:
# data['totalDistance'] = data['rideDistance'] + data['swimDistance'] + data['walkDistance']
train = train.drop(['swimDistance', 'headshotKills', 'killPoints', 'matchDuration', 'rankPoints', 'teamKills', 'winPoints'], axis=1)

train = train.groupby('matchId')
train_groupwise = train.apply(match_normalize)

In [ ]:
train_matchId = list(train.groups.keys())

split = train_groupwise.index.get_loc(train_matchId[round(len(train_matchId) * 0.8)]).start

train_X = train_groupwise.drop('winPlacePerc', axis=1).iloc[:split]
train_Y = train_groupwise['winPlacePerc'].iloc[:split]

valid_X = train_groupwise.drop('winPlacePerc', axis=1).iloc[split:]
valid_Y = train_groupwise['winPlacePerc'].iloc[split:]

In [ ]:
model = lightgbm.LGBMRegressor()
model.fit(train_X, train_Y, eval_set=(valid_X, valid_Y), eval_metric='l1')

In [ ]:
test = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/test_V2.csv')

In [ ]:
test = test.drop(['swimDistance', 'headshotKills', 'killPoints', 'matchDuration', 'rankPoints', 'teamKills', 'winPoints'], axis=1)

test = test.groupby('matchId')
test_groupwise = test.apply(match_normalize)

In [ ]:
predict_groupwise = model.predict(test_groupwise)

In [ ]:
submission = pd.read_csv('/kaggle/input/pubg-finish-placement-prediction/sample_submission_V2.csv')
submission = submission.astype({'winPlacePerc': 'float64'})

In [ ]:
for test_matchId, test_indices in test.groups.items(): 
    loc = test_groupwise.index.get_loc(test_matchId)
    groups = [index[1] for index in test_groupwise.index.values[loc]]
    t_order = np.array(groups)[predict_groupwise[loc].argsort()].tolist()
    p_orders = [kill[1]['groupId'].values[kill[1]['killPlace'].values.argsort()[::-1]].tolist() for kill in test.get_group(test_matchId).groupby('kills')]
    order = rearrange(t_order, p_orders)
    
    maxPlace = test.get_group(test_matchId).iloc[0]['maxPlace']
    numGroups = test.get_group(test_matchId).iloc[0]['numGroups']
    numVirtual = maxPlace - numGroups
    for i in range(numVirtual):
        order.insert(int(round(numGroups / (numVirtual + 1) * (i + 1) + i)), 'v')

    for i in range(len(test_indices)):
        submission.at[test_indices[i], 'winPlacePerc'] = order.index(test.get_group(test_matchId).iloc[i]['groupId']) / float(maxPlace - 1)

In [ ]:
submission.to_csv('submission.csv', index=False)